# DEVNET-2449 Python For the Enterprise Automation Exloration

In [1]:
HOST = '172.20.20.10'
PORT = 830
USER = 'vagrant'
PASS = 'vagrant'

In [2]:
import jtextfsm as textfsm
from netmiko import ConnectHandler
import pprint

def get_show_inventory(ip, username, password, enable_secret):
    # establish a connection to the device
    ssh_connection = ConnectHandler(
        device_type='cisco_ios',
        ip=ip,
        username=username,
        password=password,
        secret=enable_secret)
    # enter enable mode
    ssh_connection.enable()
    # prepend the command prompt to the result (used to identify the local device)
    result = ssh_connection.find_prompt() + "\n"
    # execute the show inventory command with extended delay
    result += ssh_connection.send_command("show inventory", delay_factor=2)
    # close SSH connection
    ssh_connection.disconnect()
    return result


In [3]:
    raw_text_data = get_show_inventory(HOST, USER, PASS, PASS)
    print ('This is what the text looks like\n' + raw_text_data)
    # Run the text through the FSM. 
    # The argument 'template' is a file handle and 'raw_text_data' is a 
    # string with the content from the show_inventory.txt file
    template = open("/home/docker/cli/show_inventory_multiple.textfsm")
    re_table = textfsm.TextFSM(template)
    fsm_results = re_table.ParseText(raw_text_data)
    # build keys for inventory sub-items 
    keys = [s for s in re_table.header]
    #build inventory list of dictionaries
    inventory = []
    for row in fsm_results:
      entry = {}
      for i, item in enumerate(row):
        entry.update({keys[i]:item})
      inventory.append(entry)
    pp = pprint.PrettyPrinter(indent=2)

This is what the text looks like
rtr1#
NAME: "Chassis", DESCR: "Cisco CSR1000V Chassis"
PID: CSR1000V          , VID: V00  , SN: 9H79916HPTZ

NAME: "module R0", DESCR: "Cisco CSR1000V Route Processor"
PID: CSR1000V          , VID: V00  , SN: JAB1303001C

NAME: "module F0", DESCR: "Cisco CSR1000V Embedded Services Processor"
PID: CSR1000V          , VID:      , SN:            




In [4]:
    print(template.read())

Value Filldown hostname (\S+)
Value name (.+)
Value description (.*)
Value productid (\S*)
Value vid (\S*)
Value Required serialnumber (\S+)

Start
  ^${hostname}[>#].*
  ^NAME: "${name}", DESCR: "${description}"
  ^PID: ${productid}.*VID: ${vid}.*SN: ${serialnumber} -> Record


In [5]:
    print('Results translated through regular expression template organized as a dictonary')
    pp.pprint(inventory) # Print full inventory as dictionary
    print([key for key, value in inventory[0].items()])  # print key values as list
    for item in inventory:   # print inventory elements a table 
        print([item[key] for key in item])
    

Results translated through regular expression template organized as a dictonary
[ { u'description': u'Cisco CSR1000V Chassis',
    u'hostname': u'rtr1',
    u'name': u'Chassis',
    u'productid': u'CSR1000V',
    u'serialnumber': u'9H79916HPTZ',
    u'vid': u'V00'},
  { u'description': u'Cisco CSR1000V Route Processor',
    u'hostname': u'rtr1',
    u'name': u'module R0',
    u'productid': u'CSR1000V',
    u'serialnumber': u'JAB1303001C',
    u'vid': u'V00'}]
[u'serialnumber', u'name', u'vid', u'description', u'hostname', u'productid']
[u'9H79916HPTZ', u'Chassis', u'V00', u'Cisco CSR1000V Chassis', u'rtr1', u'CSR1000V']
[u'JAB1303001C', u'module R0', u'V00', u'Cisco CSR1000V Route Processor', u'rtr1', u'CSR1000V']


In [6]:
    sought_product = 'CSR'
    resultlist = [d for d in inventory if d.get('productid', '') == sought_product]
    print('\nFound Devices with specific productid ' + sought_product)
    pp.pprint(resultlist) #print list of devices with sought productid
    #print ([(item['hostname'], item['productid']) for item in resultlist])  #print found items as a tuple


Found Devices with specific productid CSR
[]


In [7]:
def get_show_ip_route(ip, username, password, enable_secret):
    # establish a connection to the device
    ssh_connection = ConnectHandler(
        device_type='cisco_ios',
        ip=ip,
        username=username,
        password=password,
        secret=enable_secret)
    # enter enable mode
    ssh_connection.enable()
    # prepend the command prompt to the result (used to identify the local device)
    result = ssh_connection.find_prompt() + "\n"
    # execute the show inventory command with extended delay
    result += ssh_connection.send_command("show ip route", delay_factor=2)
    # close SSH connection
    ssh_connection.disconnect()
    return result

In [8]:
raw_text_data = get_show_ip_route(HOST, USER, PASS, PASS)
print ('This is what the text looks like\n' + raw_text_data)

This is what the text looks like
rtr1#
Codes: L - local, C - connected, S - static, R - RIP, M - mobile, B - BGP
       D - EIGRP, EX - EIGRP external, O - OSPF, IA - OSPF inter area 
       N1 - OSPF NSSA external type 1, N2 - OSPF NSSA external type 2
       E1 - OSPF external type 1, E2 - OSPF external type 2
       i - IS-IS, su - IS-IS summary, L1 - IS-IS level-1, L2 - IS-IS level-2
       ia - IS-IS inter area, * - candidate default, U - per-user static route
       o - ODR, P - periodic downloaded static route, H - NHRP, l - LISP
       a - application route
       + - replicated route, % - next hop override, p - overrides from PfR

Gateway of last resort is 10.0.2.2 to network 0.0.0.0

S*    0.0.0.0/0 [254/0] via 10.0.2.2
      10.0.0.0/8 is variably subnetted, 2 subnets, 2 masks
C        10.0.2.0/24 is directly connected, GigabitEthernet1
L        10.0.2.15/32 is directly connected, GigabitEthernet1
      172.16.0.0/16 is variably subnetted, 2 subnets, 2 masks
C        172.16.

In [15]:
import ncclient.manager
import xml.dom.minidom
from ncclient.operations import TimeoutExpiredError
  
route_filter = '''
  <routing-state xmlns="urn:ietf:params:xml:ns:yang:ietf-routing">
    <routing-instance>
      <ribs>
        <rib>
          <name>ipv4-default</name>
          <address-family xmlns:v4ur="urn:ietf:params:xml:ns:yang:ietf-ipv4-unicast-routing">v4ur:ipv4-unicast</address-family>
          <default-rib>true</default-rib>
          <routes/>
        </rib>
      </ribs>
    </routing-instance>
  </routing-state>
  '''
nckwargs = dict(
    host=HOST,
    port=PORT,
    hostkey_verify=False,
    username=USER,
    password=PASS,
    device_params={'name':"csr"}
)
m = ncclient.manager.connect(**nckwargs)  
try:
    print ('Here we are printing the RIB as XML\n')
    c = m.get(filter=('subtree', route_filter))
    xmlDom = xml.dom.minidom.parseString(str(c))
    print (xmlDom.toprettyxml( indent = " " ))
except TimeoutExpiredError as e:
    print("Operation timeout!")
except Exception as e:
    print("ERORR severity={}, tag={}".format(e.severity, e.tag))

Here we are printing the RIB as XML

<?xml version="1.0" ?>
<rpc-reply message-id="urn:uuid:b5904c5d-b24d-465f-a663-160baa7ab2ba" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
 <data>
  <routing-state xmlns="urn:ietf:params:xml:ns:yang:ietf-routing">
   <routing-instance>
    <ribs>
     <rib>
      <name>ipv4-default</name>
      <address-family>v4ur:ipv4-unicast</address-family>
      <default-rib>true</default-rib>
      <routes>
       <route>
        <destination-prefix>0.0.0.0/0</destination-prefix>
        <route-preference>254</route-preference>
        <metric>0</metric>
        <next-hop>
         <next-hop-address>10.0.2.2</next-hop-address>
        </next-hop>
        <source-protocol>static</source-protocol>
       </route>
       <route>
        <destination-prefix>10.0.2.0/24</destination-prefix>
        <route-preference>0</route-preference>
        <metric>0</metric>
        <next-hop>
         <outgoing-interface>G

In [16]:
ns = 'urn:ietf:params:xml:ns:yang:ietf-routing'
nsmap = dict(rt=ns)
route_table=[]
routes = c.data.findall(".//{%s}route" % ns)
for i, route in enumerate(routes):
    dest = route.xpath('./rt:destination-prefix/text()', namespaces=nsmap)
    hop = route.xpath('./rt:next-hop/rt:next-hop-address/text()', namespaces=nsmap)
    intfc = route.xpath('./rt:next-hop/rt:outgoing-interface/text()', namespaces=nsmap)
    prot = route.xpath('./rt:source-protocol/text()', namespaces=nsmap)
    if len(hop) > 0:
        entry = {'hop':hop[0]}
    if len(intfc) > 0:
        entry = {'intfc':intfc[0]}  
    entry.update({'dest':dest[0], 'prot':prot[0]}) 
    route_table.append(entry)
print ('\nOr we can just grab the information that is interesting\n')
pp = pprint.PrettyPrinter(indent=2)
pp.pprint(route_table)  


Or we can just grab the information that is interesting

[ { 'dest': '0.0.0.0/0', 'hop': '10.0.2.2', 'prot': 'static'},
  { 'dest': '10.0.2.0/24', 'intfc': 'GigabitEthernet1', 'prot': 'direct'},
  { 'dest': '10.0.2.15/32', 'intfc': 'GigabitEthernet1', 'prot': 'direct'},
  { 'dest': '172.16.100.0/24', 'intfc': 'GigabitEthernet3', 'prot': 'direct'},
  { 'dest': '172.16.100.10/32', 'intfc': 'GigabitEthernet3', 'prot': 'direct'},
  { 'dest': '172.20.20.0/24', 'intfc': 'GigabitEthernet2', 'prot': 'direct'},
  { 'dest': '172.20.20.10/32', 'intfc': 'GigabitEthernet2', 'prot': 'direct'},
  { 'dest': '192.168.0.0/24', 'hop': '172.20.20.100', 'prot': 'static'},
  { 'dest': '192.168.1.0/24', 'hop': '172.20.20.100', 'prot': 'static'},
  { 'dest': '192.168.2.0/24', 'hop': '172.20.20.100', 'prot': 'static'},
  { 'dest': '192.168.4.0/24', 'hop': '172.20.20.100', 'prot': 'static'},
  { 'dest': '198.18.10.0/24', 'hop': '172.20.20.100', 'prot': 'static'},
  { 'dest': '198.18.11.0/24', 'hop': '172.20.20

In [23]:
cpu_util = '''<filter>
<cpu-usage xmlns="urn:cisco:params:xml:ns:yang:cisco-process-cpu">
<cpu-utilization>
</cpu-utilization>
</cpu-usage></filter>'''

cpu_process = '''<filter>
<cpu-usage xmlns="urn:cisco:params:xml:ns:yang:cisco-process-cpu" xmlns:y="http://tail-f.com/ns/rest" xmlns:pcpu="urn:cisco:params:xml:ns:yang:cisco-process-cpu">
<process-cpu-usage>
</process>
</process-cpu-usage>
</filter>'''
print ('Using NETCONF, you can present data as XML without any conversion')
cpu_util = str( m.get( filter=cpu_util ) )
xmlDom = xml.dom.minidom.parseString(cpu_util)
print xmlDom.toprettyxml( indent = " " )

Using NETCONF, you can present data as XML without any conversion
<?xml version="1.0" ?>
<rpc-reply message-id="urn:uuid:8da51ccc-3bae-4eaa-9aa6-792df8ebcd9d" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
 <data>
  <cpu-usage xmlns="urn:cisco:params:xml:ns:yang:cisco-process-cpu">
   <cpu-utilization>
    <five-seconds>0</five-seconds>
    <five-seconds-intr>0</five-seconds-intr>
    <one-minute>0</one-minute>
    <five-minutes>0</five-minutes>
   </cpu-utilization>
  </cpu-usage>
 </data>
</rpc-reply>



In [24]:
import xmltodict
print ('Or you can convert to Dictionary')
cpu_dict = xmltodict.parse( cpu_util )['rpc-reply']['data']
print [(i,v) for i,v in cpu_dict['cpu-usage']['cpu-utilization'].iteritems()]

Or you can convert to Dictionary
[(u'five-seconds', u'0'), (u'five-seconds-intr', u'0'), (u'one-minute', u'0'), (u'five-minutes', u'0')]


In [29]:
print ('\nWith structured data you can filter or sort \nHere we are grabbing the processes with higher run-time')
cpu_dict = xmltodict.parse( str( m.get( filter=cpu_process ) ) )
cpu_dict = cpu_dict['rpc-reply']['data']
nonzero_process = [t for t in cpu_dict['cpu-usage']['process-cpu-usage']['process'] if int(t['total-run-time']) > 100]
for i in nonzero_process: print (i['name'],i['pid'],i['total-run-time'])


With structured data you can filter or sort 
Here we are grabbing the processes with higher run-time


TypeError: 'module' object is not callable

In [20]:
CONF_FILTER ='''<filter>
  <interfaces xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces">
    <interface></interface>
  </interfaces>
</filter>'''
STATE_FILTER ='''<filter>
  <interfaces-state xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces">
    <interface></interface>
  </interfaces-state>
</filter>'''
print('\nHere is the XML data for Interfaces Operational State')
interfaces = m.get(STATE_FILTER)
pp = pprint.PrettyPrinter(indent=2)
print(xml.dom.minidom.parseString(interfaces.xml).toprettyxml(indent=" "))


Here is the XML data for Interfaces Operational State
<?xml version="1.0" ?>
<rpc-reply message-id="urn:uuid:bf2c9fc4-3252-4d30-a59a-aa678507af8e" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
 <data>
  <interfaces-state xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces">
   <interface>
    <name>GigabitEthernet1</name>
    <oper-status>up</oper-status>
    <phys-address>08:00:27:59:45:4f</phys-address>
    <speed>1000000000</speed>
    <statistics>
     <in-octets>43885</in-octets>
     <in-broadcast-pkts>0</in-broadcast-pkts>
     <in-multicast-pkts>0</in-multicast-pkts>
     <in-discards>0</in-discards>
     <in-errors>0</in-errors>
     <out-octets>59217</out-octets>
     <out-discards>0</out-discards>
     <out-errors>0</out-errors>
     <in-pkts xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces-ext">442</in-pkts>
     <out-pkts xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces-ext">345</out-pkts>
    </statistics>
   </int

In [19]:
m = ncclient.manager.connect(**nckwargs)
print('\nHere is the XML data for Interfaces Configuration')
interfaces = m.get_config('running',CONF_FILTER)
print(xml.dom.minidom.parseString(interfaces.xml).toprettyxml(indent=" "))


Here is the XML data for Interfaces Configuration
<?xml version="1.0" ?>
<rpc-reply message-id="urn:uuid:4aa6905a-8e64-48a1-8fcf-e360f570fdb3" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
 <data>
  <interfaces xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces">
   <interface>
    <name>GigabitEthernet1</name>
    <type xmlns:ianaift="urn:ietf:params:xml:ns:yang:iana-if-type">ianaift:ethernetCsmacd</type>
    <enabled>true</enabled>
    <ipv4 xmlns="urn:ietf:params:xml:ns:yang:ietf-ip"/>
    <ipv6 xmlns="urn:ietf:params:xml:ns:yang:ietf-ip"/>
   </interface>
   <interface>
    <name>GigabitEthernet2</name>
    <type xmlns:ianaift="urn:ietf:params:xml:ns:yang:iana-if-type">ianaift:ethernetCsmacd</type>
    <enabled>true</enabled>
    <ipv4 xmlns="urn:ietf:params:xml:ns:yang:ietf-ip">
     <address>
      <ip>172.20.20.10</ip>
      <netmask>255.255.255.0</netmask>
     </address>
    </ipv4>
    <ipv6 xmlns="urn:ietf:params:xml:ns:

In [16]:
from subprocess import Popen, PIPE, STDOUT
SCHEMA_TO_GET = 'ietf-interfaces'
print(' We are connecting to the device and downloading the YANG Module')
print("Now we will do a 'pyang -f tree of %s.yang'" % SCHEMA_TO_GET)
print('This structure is what data we can get (ro/rw) or set (rw)')
c = m.get_schema(SCHEMA_TO_GET)
p = Popen(['pyang', '-f', 'tree'], stdout=PIPE, stdin=PIPE, stderr=PIPE)
stdout_data = p.communicate(input=c.data)[0]
print stdout_data

 We are connecting to the device and downloading the YANG Module
Now we will do a 'pyang -f tree of ietf-interfaces.yang'
This structure is what data we can get (ro/rw) or set (rw)
module: ietf-interfaces
   +--rw interfaces
   |  +--rw interface* [name]
   |     +--rw name                        string
   |     +--rw description?                string
   |     +--rw type                        identityref
   |     +--rw enabled?                    boolean
   |     +--rw link-up-down-trap-enable?   enumeration {if-mib}?
   +--ro interfaces-state
      +--ro interface* [name]
         +--ro name               string
         +--ro type               identityref
         +--ro admin-status       enumeration {if-mib}?
         +--ro oper-status        enumeration
         +--ro last-change?       yang:date-and-time
         +--ro if-index           int32 {if-mib}?
         +--ro phys-address?      yang:phys-address
         +--ro higher-layer-if*   interface-state-ref
         +--ro lower

In [40]:
from ydk.services import CRUDService
from ydk.providers import NetconfServiceProvider
from ydk.models.ietf import ietf_interfaces as intf

# create NETCONF provider
provider = NetconfServiceProvider(address=HOST,
                                  port=PORT,
                                  username=USER,
                                  password=PASS)

# create CRUD service
crud = CRUDService()

# query object
q_i = intf.Interfaces()

# get stuff
intfs = crud.read(provider, q_i)

# print interface names and types
for i in intfs.interface:
    print('%s, %s, %s' % (i.name, i.type._meta_info().yang_name, i.description))

GigabitEthernet1, ethernetCsmacd, None
GigabitEthernet2, ethernetCsmacd, None
GigabitEthernet3, ethernetCsmacd, None
Loopback999, softwareLoopback, None
Tunnel4, tunnel, None


In [46]:
q_i = intf.InterfacesState()

# get stuff
intfs = crud.read(provider, q_i)
q_i = intf.InterfacesState()

# get stuff
intfs = crud.read(provider, q_i)
int_info=[(i.name, i.statistics.out_pkts, int(i.speed)/1000000, i.oper_status) for i in intfs.interface]
for thing in int_info: print(str(thing))

('GigabitEthernet1', 345, 1000, <OperStatusEnum.up: 1>)
('GigabitEthernet2', 3315, 1000, <OperStatusEnum.up: 1>)
('GigabitEthernet3', 4812, 10000, <OperStatusEnum.up: 1>)
('Loopback999', 0, 8000, <OperStatusEnum.up: 1>)
('Tunnel4', 0, 0, <OperStatusEnum.down: 2>)


# Create Software Loopback

In [48]:
# new interface
new_loopback = intf.Interfaces.Interface()

# create a new loopback
new_loopback.name = 'Loopback666'
new_loopback.type = iftype.SoftwareloopbackIdentity()
new_loopback.description = 'Created by DevNet2445'
res = crud.create(provider, new_loopback)

NameError: name 'SoftwareloopbackIdentity' is not defined

In [21]:
# get stuff
intfs = crud.read(provider, q_i)

for i in intfs.interface:
    print('%s, %s, %s' % (i.name, i.type._meta_info().yang_name, i.description))

GigabitEthernet1, ethernetCsmacd, None
GigabitEthernet2, ethernetCsmacd, None
GigabitEthernet3, ethernetCsmacd, None
Tunnel4, tunnel, None


# Delete Software Loopback

In [22]:
# create CRUD service
crud = CRUDService()

# interface to delete
to_del = intf.Interfaces.Interface()

# create a new loopback
to_del.name = 'Loopback666'
res = crud.delete(provider, to_del)

YPYServiceProviderError: Server rejected request.
	error-type: application
	error-tag: data-missing
	error-severity: error
	error-path: /nc:rpc/nc:edit-config/nc:config/if:interfaces/if:interface[if:name='Loopback666']
	bad-element: interface

In [14]:
# get stuff
intfs = crud.read(provider, q_i)

for i in intfs.interface:
    print('%s, %s, %s' % (i.name, i.type._meta_info().yang_name, i.description))
provider.close()

GigabitEthernet1, ethernetCsmacd, None
GigabitEthernet2, ethernetCsmacd, None
GigabitEthernet3, ethernetCsmacd, None
Tunnel4, tunnel, None
